<a href="https://colab.research.google.com/github/sportlosos/desktop-tutorial/blob/main/Chunks_Whisper_%2B_GPT_3_5_for_long_inputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Transcription  with OpenAI's Whisper and and comletion with GPT 3.5 Turbo **


Edited by Zlata Poni # pragueschool.media mediaschool.ai
20.03.24


Original Whisper  [![original_notebook shield](https://img.shields.io/static/v1?label=&message=Notebook&color=blue&style=for-the-badge&logo=googlecolab&link=https://colab.research.google.com/github/ArthurFDLR/whisper-youtube/blob/main/whisper_youtube.ipynb)](https://colab.research.google.com/github/ArthurFDLR/whisper-youtube/blob/main/whisper_youtube.ipynb)
[![repository shield](https://img.shields.io/static/v1?label=&message=Repository&color=blue&style=for-the-badge&logo=github&link=https://github.com/openai/whisper)](https://github.com/openai/whisper)


What for:  Get minnutes from metings, edit long texts into social media formats, make sence of lengthy videos and recordings.

* ## PART 1. Transcribe  
> **#(skip, if working with a text input)**
-------------------------------------------------------------------

Whisper is a general-purpose speech recognition model. It is trained on a large dataset of diverse audio and is also a multi-task model that can perform multilingual speech recognition as well as speech translation and language identification.

You'll be able to explore most inference parameters or use the Notebook as-is to store the transcript and video audio in your Google Drive.
>
>

* ## PART 2.  Apply GPT 3.5 turbo to your transcript
------------------------------------------------------------
# **Provide your openai  API key  and operate on lengthy inputs by sequentially applying   PROMPTs to chunks of the text**

#  info@pragueschool.media |  @sportlosos - Telegram |@possuminside Discord

** MIT License
----------------------------------------------------------------


Copyright (c) 2022 OpenAI

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.


# PART 1. Whisper

In [ ]:
#@markdown # **1.1Connect your G-Drive** 🏗️
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@markdown # **1.2Check GPU type** 🕵️


#@markdown The type of GPU you get assigned in your Colab session defined the speed at which the video will be transcribed.
#@markdown The higher the number of floating point operations per second (FLOPS), the faster the transcription.
#@markdown But even the least powerful GPU available in Colab is able to run any Whisper model.
#@markdown Make sure you've selected `GPU` as hardware accelerator for the Notebook (Runtime &rarr; Change runtime type &rarr; Hardware accelerator).

#@markdown |  GPU   |  GPU RAM   | FP32 teraFLOPS |     Availability   |
#@markdown |:------:|:----------:|:--------------:|:------------------:|
#@markdown |  T4    |    16 GB   |       8.1      |         Free       |
#@markdown | P100   |    16 GB   |      10.6      |      Colab Pro     |
#@markdown | V100   |    16 GB   |      15.7      |  Colab Pro (Rare)  |

#@markdown ---
#@markdown **Factory reset your Notebook's runtime if you want to get assigned a new GPU.**

!nvidia-smi -L

!nvidia-smi

In [ ]:
#@markdown # **1.3Install libraries** 🏗️
#@markdown This cell will take a little while to download several libraries, including Whisper.

#@markdown ---

! pip install git+https://github.com/openai/whisper.git
! pip install yt-dlp

import sys
import warnings
import whisper
from pathlib import Path
import yt_dlp
import subprocess
import torch
import shutil
import numpy as np
from IPython.display import display, Markdown, YouTubeVideo

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

In [ ]:
#@markdown # **1.4Save data in Google Drive** 💾
#@markdown Enter a Google Drive path and run this cell if you want to store the results inside Google Drive. This creates a folder "Medialist" on your Google Drive, where you will find the results of Whisper transcription and  GPT3.5 completion
#@markdown This will create an output folder MEDIALIST  on your G-riive.
#@markdown ---
#@markdown **Google Drive Path:**
drive_path = "/content/drive/MyDrive/Medialist" #@param {type:"string"}
#@markdown ---
#@markdown **Note:** Run this cell again if you change your Google Drive path.

# Code to mount Google Drive and ensure the specified folder exists
from google.colab import drive
from pathlib import Path

# Mount Google Drive
drive_mount_path = '/content/drive'
drive.mount(drive_mount_path, force_remount=True)  # Ensures Google Drive is accessible

# Define the full path to the target folder in Google Drive
drive_whisper_path = Path(drive_path)

# Ensure the target folder exists, create it if it doesn't
if not drive_whisper_path.is_dir():
    drive_whisper_path.mkdir(parents=True, exist_ok=True)
    print(f"Folder '{drive_whisper_path}' created.")
else:
    print(f"Folder '{drive_whisper_path}' already exists.")

In [ ]:
#@markdown # **1.5Model selection** 🧠

#@markdown As of the first public release, there are 4 pre-trained options to play with:

#@markdown |  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
#@markdown |:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
#@markdown |  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~1 GB     |      ~32x      |
#@markdown |  base  |    74 M    |     `base.en`      |       `base`       |     ~1 GB     |      ~16x      |
#@markdown | small  |   244 M    |     `small.en`     |      `small`       |     ~2 GB     |      ~6x       |
#@markdown | medium |   769 M    |    `medium.en`     |      `medium`      |     ~5 GB     |      ~2x       |
#@markdown | large  |   1550 M   |        N/A         |      `large`       |    ~10 GB     |       1x       |

#@markdown ---
Model = 'medium' #@param ['tiny.en', 'tiny', 'base.en', 'base', 'small.en', 'small', 'medium.en', 'medium', 'large']
#@markdown ---
#@markdown **Run this cell again if you change the model.**

whisper_model = whisper.load_model(Model)

if Model in whisper.available_models():
    display(Markdown(
        f"**{Model} model is selected.**"
    ))
else:
    display(Markdown(
        f"**{Model} model is no longer available.**<br /> Please select one of the following:<br /> - {'<br /> - '.join(whisper.available_models())}"
    ))

In [ ]:
#@markdown # **1.6File selection** 📺

#@markdown Enter the URL of the Youtube video you want to transcribe, whether you want to save the audio file in your Google Drive, and run the cell.

Type = "Youtube video or playlist" #@param ['Youtube video or playlist', 'Google Drive']
#@markdown ---
#@markdown #### **Youtube video or playlist**
URL = "https://www.youtube.com/watch?v=vEd-LqBCONg" #@param {type:"string"}
# store_audio = True #@param {type:"boolean"}
#@markdown ---
#@markdown #### **Google Drive video, audio (mp4, wav), or folder containing video and/or audio files**
video_path = "" #@param {type:"string"}
#@markdown ---
#@markdown **Run this cell again if you change the video.**

from pathlib import Path
import yt_dlp
import shutil
import subprocess
from IPython.display import Markdown
from pathlib import Path

video_path_local_list = []

if Type == "Youtube video or playlist":
    ydl_opts = {
        'format': 'm4a/bestaudio/best',
        'outtmpl': '%(id)s.%(ext)s',
        'postprocessors': [{  # Extract audio using ffmpeg
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
        }]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        error_code = ydl.download([URL])
        list_video_info = [ydl.extract_info(URL, download=False)]

    for video_info in list_video_info:
        video_path_local_list.append(Path(f"{video_info['id']}.wav"))

elif Type == "Google Drive":
    # Direct use of the provided video_path assuming it's a full path
    corrected_video_path = Path(video_path)
    if corrected_video_path.is_dir():
        for video_path_drive in corrected_video_path.glob("**/*"):
            if video_path_drive.is_file():
                display(Markdown(f"**{str(video_path_drive)} selected for transcription.**"))
            elif video_path_drive.is_dir():
                display(Markdown(f"**Subfolders not supported.**"))
            else:
                display(Markdown(f"**{str(video_path_drive)} does not exist, skipping.**"))
            video_path_local = Path(".").resolve() / video_path_drive.name
            shutil.copy(video_path_drive, video_path_local)
            video_path_local_list.append(video_path_local)
    elif corrected_video_path.is_file():
        video_path_local = Path(".").resolve() / corrected_video_path.name
        shutil.copy(corrected_video_path, video_path_local)
        video_path_local_list.append(video_path_local)
        display(Markdown(f"**{str(corrected_video_path)} selected for transcription.**"))
    else:
        display(Markdown(f"**{str(corrected_video_path)} does not exist.**"))

else:
    raise(TypeError("Please select a supported input type."))

for video_path_local in video_path_local_list:
    if video_path_local.suffix == ".mp4":
        video_path_local = video_path_local.with_suffix(".wav")
        result = subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])


In [ ]:
#@markdown # **1.7Run Whisper - get Transcript** 🚀

#@markdown Run this cell to execute the transcription of the video. This can take a while and vary based on the length of the video and the number of parameters of the model selected above.

#@markdown ## **Parameters** ⚙️

#@markdown ### **Behavior control**
#@markdown ---
language = "English" #@param ['Auto detection', 'Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Assamese', 'Azerbaijani', 'Bashkir', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Breton', 'Bulgarian', 'Burmese', 'Castilian', 'Catalan', 'Chinese', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Estonian', 'Faroese', 'Finnish', 'Flemish', 'French', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hungarian', 'Icelandic', 'Indonesian', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Lao', 'Latin', 'Latvian', 'Letzeburgesch', 'Lingala', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Moldavian', 'Moldovan', 'Mongolian', 'Myanmar', 'Nepali', 'Norwegian', 'Nynorsk', 'Occitan', 'Panjabi', 'Pashto', 'Persian', 'Polish', 'Portuguese', 'Punjabi', 'Pushto', 'Romanian', 'Russian', 'Sanskrit', 'Serbian', 'Shona', 'Sindhi', 'Sinhala', 'Sinhalese', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog', 'Tajik', 'Tamil', 'Tatar', 'Telugu', 'Thai', 'Tibetan', 'Turkish', 'Turkmen', 'Ukrainian', 'Urdu', 'Uzbek', 'Valencian', 'Vietnamese', 'Welsh', 'Yiddish', 'Yoruba']
#@markdown > Language spoken in the audio, use `Auto detection` to let Whisper detect the language.
#@markdown ---
verbose = 'Live transcription' #@param ['Live transcription', 'Progress bar', 'None']
#@markdown > Whether to print out the progress and debug messages.
#@markdown ---
output_format = 'all' #@param ['txt', 'vtt', 'srt', 'tsv', 'json', 'all']
#@markdown > Type of file to generate to record the transcription.
#@markdown ---
task = 'transcribe' #@param ['transcribe', 'translate']
#@markdown > Whether to perform X->X speech recognition (`transcribe`) or X->English translation (`translate`).
#@markdown ---

#@markdown <br/>

#@markdown ### **Optional: Fine tunning**
#@markdown ---
temperature = 0.1 #@param {type:"slider", min:0, max:1, step:0.05}
#@markdown > Temperature to use for sampling.
#@markdown ---
temperature_increment_on_fallback = 0.15 #@param {type:"slider", min:0, max:1, step:0.05}
#@markdown > Temperature to increase when falling back when the decoding fails to meet either of the thresholds below.
#@markdown ---
best_of = 8 #@param {type:"integer"}
#@markdown > Number of candidates when sampling with non-zero temperature.
#@markdown ---
beam_size = 8 #@param {type:"integer"}
#@markdown > Number of beams in beam search, only applicable when temperature is zero.
#@markdown ---
patience = 1.0 #@param {type:"number"}
#@markdown > Optional patience value to use in beam decoding, as in [*Beam Decoding with Controlled Patience*](https://arxiv.org/abs/2204.05424), the default (1.0) is equivalent to conventional beam search.
#@markdown ---
length_penalty = -0.05 #@param {type:"slider", min:-0.05, max:1, step:0.05}
#@markdown > Optional token length penalty coefficient (alpha) as in [*Google's Neural Machine Translation System*](https://arxiv.org/abs/1609.08144), set to negative value to uses simple length normalization.
#@markdown ---
suppress_tokens = "-1" #@param {type:"string"}
#@markdown > Comma-separated list of token ids to suppress during sampling; '-1' will suppress most special characters except common punctuations.
#@markdown ---
initial_prompt = "Transcript of youtube video" #@param {type:"string"}
#@markdown > Optional text to provide as a prompt for the first window.
#@markdown ---
condition_on_previous_text = True #@param {type:"boolean"}
#@markdown > if True, provide the previous output of the model as a prompt for the next window; disabling may make the text inconsistent across windows, but the model becomes less prone to getting stuck in a failure loop.
#@markdown ---
fp16 = True #@param {type:"boolean"}
#@markdown > whether to perform inference in fp16.
#@markdown ---
compression_ratio_threshold = 2.4 #@param {type:"number"}
#@markdown > If the gzip compression ratio is higher than this value, treat the decoding as failed.
#@markdown ---
logprob_threshold = -1.0 #@param {type:"number"}
#@markdown > If the average log probability is lower than this value, treat the decoding as failed.
#@markdown ---
no_speech_threshold = 0.6 #@param {type:"slider", min:-0.0, max:1, step:0.05}
#@markdown > If the probability of the <no_speech> token is higher than this value AND the decoding has failed due to `logprob_threshold`, consider the segment as silence.
#@markdown ---

import numpy as np
import shutil
import whisper
import warnings
from IPython.display import Markdown

verbose_lut = {
    'Live transcription': True,
    'Progress bar': False,
    'None': None
}

args = dict(
    language = (None if language == "Auto detection" else language),
    verbose = verbose_lut[verbose],
    task = task,
    temperature = temperature,
    temperature_increment_on_fallback = temperature_increment_on_fallback,
    best_of = best_of,
    beam_size = beam_size,
    patience=patience,
    length_penalty=(length_penalty if length_penalty>=0.0 else None),
    suppress_tokens=suppress_tokens,
    initial_prompt=(None if not initial_prompt else initial_prompt),
    condition_on_previous_text=condition_on_previous_text,
    fp16=fp16,
    compression_ratio_threshold=compression_ratio_threshold,
    logprob_threshold=logprob_threshold,
    no_speech_threshold=no_speech_threshold
)

temperature = args.pop("temperature")
temperature_increment_on_fallback = args.pop("temperature_increment_on_fallback")
if temperature_increment_on_fallback is not None:
    temperature = tuple(np.arange(temperature, 1.0 + 1e-6, temperature_increment_on_fallback))
else:
    temperature = [temperature]

if Model.endswith(".en") and args["language"] not in {"en", "English"}:
    warnings.warn(f"{Model} is an English-only model but received '{args['language']}'; using English instead.")
    args["language"] = "en"

for video_path_local in video_path_local_list:
    display(Markdown(f"### {video_path_local}"))

    video_transcription = whisper.transcribe(
        whisper_model,
        str(video_path_local),
        temperature=temperature,
        **args,
    )

    # Save output
    whisper.utils.get_writer(
        output_format=output_format,
        output_dir=video_path_local.parent
    )(
        video_transcription,
        str(video_path_local.stem),
        options=dict(
            highlight_words=False,
            max_line_count=None,
            max_line_width=None,
        )
    )

    def exportTranscriptFile(ext: str):
        local_path = video_path_local.parent / video_path_local.with_suffix(ext)
        export_path = drive_whisper_path / video_path_local.with_suffix(ext)

        # Check if the source and destination files are the same
        if local_path != export_path:
            shutil.copy(local_path, export_path)
            display(Markdown(f"**Transcript file created: {export_path}**"))
        else:
            display(Markdown(f"**{local_path} already exists, skipping copy.**"))

    if output_format == "all":
        for ext in ('.txt', '.vtt', '.srt', '.tsv', '.json'):
            exportTranscriptFile(ext)
    else:
        exportTranscriptFile("." + output_format)

    def copy_transcript_to_drive(local_folder_path, drive_folder_path, extensions):
        for ext in extensions:
          for local_file in local_folder_path.glob(f'*{ext}'):
            drive_file_path = drive_folder_path / local_file.name
            shutil.copy(local_file, drive_file_path)
            print(f"File {local_file.name} copy {drive_file_path}")

# file extentions
extensions = ['.txt', '.vtt', '.srt', '.tsv', '.json']

# call function "copy" file
# change 'local_output_folder_path' folder path, where transcript will be saved
local_output_folder_path = Path('/content')  # path examplе, temp folder
copy_transcript_to_drive(local_output_folder_path, drive_whisper_path, extensions)



* ##  **PART 2** ⏰  Working on your transcript with GPT 3.5 model,  here you will add prompts to perform summarisation on the chunks of the transcript

In [ ]:
#@markdown # **2.1 Make sure the G-rive is intact** 📺
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@markdown ###  **Installation**
!pip install openai==0.28
!pip install OpenAI



In [ ]:
#@markdown # **2.3Configuration Section: provide A- openai key, B-length of chunk, C - path to the file**

#@markdown #### You can generate your OPENAI API KEY here - https://platform.openai.com/api-keys
OPENAI_API_KEY = '' #@param {type:"string"}
#@markdown ###### CHUNK lengh defines How many words GPT 3.5 will process at once. Depends on the complexity of your prompt and language, 750 is a good start for a test.
WORDS_PER_CHUNK = 750 #@param {type:"integer"}
 #@markdown ### You can find the path at the end of your transcription. If you have another document to chunk and prompt -  put it on your G-rive first, any txt/srt !!! NO gdoc
CUSTOM_TRANSCRIPT_PATH = ''  #@param {type:"string"}
drive_path = CUSTOM_TRANSCRIPT_PATH

#@markdown ---
from google.colab import drive
from pathlib import Path

# Mount Google Drive
drive_mount_path = '/content/drive'
drive.mount(drive_mount_path, force_remount=True)  # Ensures Google Drive is accessible


# Assuming the variable drive_path has already been defined in your code
transcript_folder_path = drive_path  # Using the already defined path to the folder containing the transcripts

# Code to find the latest transcript file or use the custom path provided by the user
def find_latest_transcript_file(folder_path, file_extension='.txt'):
    try:
        folder = Path(folder_path)
        files = list(folder.glob(f'*{file_extension}'))
        latest_file = max(files, key=os.path.getctime)
        return latest_file
    except ValueError:
        print(f"No files with extension {file_extension} found in {folder_path}.")
        return None

if CUSTOM_TRANSCRIPT_PATH:
    TRANSCRIPT_PATH =  drive_path
    print(f"Using provided transcript file: {TRANSCRIPT_PATH}")
else:
    latest_transcript_file = find_latest_transcript_file(transcript_folder_path)
    if latest_transcript_file:
        TRANSCRIPT_PATH = str(latest_transcript_file)  # Convert the Path object to a string
        print(f"Using latest transcript file: {TRANSCRIPT_PATH}")
    else:
        print("Failed to find the latest transcript file. Please check the folder path and try again.")


In [ ]:

# Initialize OpenAI API key
import os
import openai
openai.api_key = OPENAI_API_KEY



#@markdown # **2.4PROMPT**
#@markdown ## Enter the prompt as you usually do. Provide /GPT 3.5 turbo/ with context of your transcription and tell what results do you expect.
CUSTOM_PROMPT = "" #@param {type:"string"}

#@markdown ### TIPS: Use your regular prompting methods - you can ask for clear and more contious version, summarisation, transcription, list etc; You can specify Role and tone,
#@markdown FOR EXAMPLE "This is a transcript of my lecture, please summarize what's been said" or "this is a transcript of meeting, please take minutes on what' been discussed and what we agreed on"
def mount_google_drive():
    """
    Mounts the Google Drive to access files stored in it.
    """
    drive.mount('/content/drive')

if __name__ == '__main__':
    mount_google_drive()
    # Add your logic here to read and process the file

def read_transcript(file_path):
    """
    Reads the transcript text from a file.

    Parameters:
    - file_path: str, path to the transcript file.

    Returns:
    - str, the content of the transcript.
    """
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None

def chunk_transcript(transcript, words_per_chunk=WORDS_PER_CHUNK):
    """
    Breaks the transcript into chunks of approximately a specified number of words.

    Parameters:
    - transcript: str, the full transcript text.
    - words_per_chunk: int, number of words per chunk.

    Returns:
    - list of str, the chunks of the transcript.
    """
    words = transcript.split()
    return [' '.join(words[i:i+words_per_chunk]) for i in range(0, len(words), words_per_chunk)]

def summarize_chunk(chunk):
    """
    Summarizes a chunk of text using OpenAI's gpt-3.5-turbo model.

    Parameters:
    - chunk: str, a chunk of transcript text to summarize.

    Returns:
    - str, the summary of the chunk.
    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": CUSTOM_PROMPT},
            {"role": "user", "content": chunk}
        ]
    )
    return response.choices[0].message['content']

def save_summaries(summaries, original_file_path):
    """
    Saves the summaries to a new file with '_OPENAI' added before the file extension in the original filename.

    Parameters:
    - summaries: list of str, the summaries to save.
    - original_file_path: str, the original path of the transcript file.
    """
    base, ext = os.path.splitext(original_file_path)
    new_file_path = f"{base}_OPENAI{ext}"

    with open(new_file_path, 'w') as file:
        for summary in summaries:
            file.write(summary + "\n\n")

    print(f"Summaries saved to {new_file_path}")

def main():
    """
    Main function to read a transcript, chunk it, summarize each chunk, and save the summaries.
    """
    mount_google_drive()
    transcript = read_transcript(TRANSCRIPT_PATH)
    if transcript:
        chunks = chunk_transcript(transcript)
        summaries = [summarize_chunk(chunk) for chunk in chunks]

        # Save the summaries to a new file
        save_summaries(summaries, TRANSCRIPT_PATH)

        # Print the summaries to the screen
        print("Summaries:\n")
        for i, summary in enumerate(summaries, start=1):
            print(f"Summary {i}:\n{summary}\n")
    else:
        print("Failed to read transcript. Please check the file path and try again.")

if __name__ == '__main__':
    main()
